How we one hot encode game state into observation state with one hot encoding.

Because TFT is a game with a high amount of categorical data, we need to one hot encode so that it can be processed by our machine for reinforcement learning.

High Cardinality Categorical Data

There are 100s of items and champions in TFT that need to be encoded. Each champion can hold 3 items and
each player can have up to 9 champions on bench and board, so the number of combination grows extremely quickly.

This is how I've tried to encode this data.

In [1]:
import json
import sys
import random
from sklearn.preprocessing import OneHotEncoder
import numpy as np
np.set_printoptions(threshold=sys.maxsize)


champions_data = json.load(open('../tft_static_data/set5patch1115/champions.json'))
items_data = json.load(open('../tft_static_data/set5patch1115/items.json'))


champion_ids = [c['championId'] for c in champions_data]
item_ids = [str(i['id']) for i in items_data]

print(champion_ids)
print("\n")
print(item_ids)

['TFT5_Aatrox', 'TFT5_Akshan', 'TFT5_Aphelios', 'TFT5_Ashe', 'TFT5_Brand', 'TFT5_Diana', 'TFT5_Draven', 'TFT5_Fiddlesticks', 'TFT5_Galio', 'TFT5_Garen', 'TFT5_Gragas', 'TFT5_Gwen', 'TFT5_Hecarim', 'TFT5_Heimerdinger', 'TFT5_Ivern', 'TFT5_Irelia', 'TFT5_Jax', 'TFT5_Kalista', 'TFT5_Karma', 'TFT5_Kayle', 'TFT5_Kennen', 'TFT5_Khazix', 'TFT5_Kled', 'TFT5_LeeSin', 'TFT5_Leona', 'TFT5_Lucian', 'TFT5_Lulu', 'TFT5_Lux', 'TFT5_MissFortune', 'TFT5_Nautilus', 'TFT5_Nidalee', 'TFT5_Nocturne', 'TFT5_Nunu', 'TFT5_Olaf', 'TFT5_Poppy', 'TFT5_Pyke', 'TFT5_Rakan', 'TFT5_Rell', 'TFT5_Riven', 'TFT5_Sejuani', 'TFT5_Senna', 'TFT5_Sett', 'TFT5_Soraka', 'TFT5_Syndra', 'TFT5_Teemo', 'TFT5_Thresh', 'TFT5_Tristana', 'TFT5_Udyr', 'TFT5_Varus', 'TFT5_Vayne', 'TFT5_Velkoz', 'TFT5_Viego', 'TFT5_Vladimir', 'TFT5_Volibear', 'TFT5_Yasuo', 'TFT5_Ziggs', 'TFT5_Zyra']


['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '22', '23', '24', '25', '26', '27', '28', '29', '33', '

In [6]:
# Mock out what the game state we want to encode will look like

# A board will consists of up to 9 champions, each with 3 item slots and their level.
# A champion is represented by a array of size 4 with [CHAMPION_ID, ITEM1, ITEM2, ITEM3, CHAMPION_LEVEL]
board = [
    [
        random.choice(champion_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.randint(1,3)
    ] for i in range(8)
]
board.append(['None',0,0,0,0]) # add an unoccupied board slot

bench = [
    [
        random.choice(champion_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.randint(1,3)
    ] for i in range(8)
]
bench.append(['None',0,0,0,0])

# SHOP just has the champion names
shop = [random.choice(champion_ids) for i in range(5)] 

print("BOARD:", board, "\n\nBENCH:", bench, "\n\nSHOP:", shop)


BOARD: [['TFT5_Akshan', '78', '34', '37', 2], ['TFT5_Vladimir', '1191', '55', '55', 2], ['TFT5_Sett', '2024', '8', '1178', 2], ['TFT5_Galio', '12', '26', '46', 1], ['TFT5_Nidalee', '55', '99', '2047', 1], ['TFT5_Gragas', '47', '2059', '2017', 3], ['TFT5_Udyr', '2045', '2039', '14', 1], ['TFT5_Khazix', '79', '58', '2069', 2], ['None', 0, 0, 0, 0]] 

BENCH: [['TFT5_Rakan', '8', '2044', '2012', 3], ['TFT5_Senna', '69', '38', '18', 1], ['TFT5_Kayle', '2077', '2017', '2035', 1], ['TFT5_Leona', '34', '1', '1196', 2], ['TFT5_Velkoz', '7', '9', '29', 3], ['TFT5_Akshan', '2025', '6', '2029', 1], ['TFT5_Teemo', '2046', '69', '4', 1], ['TFT5_Udyr', '2047', '6', '25', 3], ['None', 0, 0, 0, 0]] 

SHOP: ['TFT5_Vladimir', 'TFT5_Yasuo', 'TFT5_Aphelios', 'TFT5_MissFortune', 'TFT5_Yasuo']


In [7]:
# Now we need to represent this game state as a 1-d vector. 
# Note that since we work with np arrays, everything is converted to 
# a string
flattened_game_state = np.hstack(shop+board+bench)
print(flattened_game_state)
# Flattened game state should be 95 items long (Shop=5, Board=45, Bench=45)
assert len(flattened_game_state) == 5 + 9*5 + 9*5  

['TFT5_Vladimir' 'TFT5_Yasuo' 'TFT5_Aphelios' 'TFT5_MissFortune'
 'TFT5_Yasuo' 'TFT5_Akshan' '78' '34' '37' '2' 'TFT5_Vladimir' '1191' '55'
 '55' '2' 'TFT5_Sett' '2024' '8' '1178' '2' 'TFT5_Galio' '12' '26' '46'
 '1' 'TFT5_Nidalee' '55' '99' '2047' '1' 'TFT5_Gragas' '47' '2059' '2017'
 '3' 'TFT5_Udyr' '2045' '2039' '14' '1' 'TFT5_Khazix' '79' '58' '2069' '2'
 'None' '0' '0' '0' '0' 'TFT5_Rakan' '8' '2044' '2012' '3' 'TFT5_Senna'
 '69' '38' '18' '1' 'TFT5_Kayle' '2077' '2017' '2035' '1' 'TFT5_Leona'
 '34' '1' '1196' '2' 'TFT5_Velkoz' '7' '9' '29' '3' 'TFT5_Akshan' '2025'
 '6' '2029' '1' 'TFT5_Teemo' '2046' '69' '4' '1' 'TFT5_Udyr' '2047' '6'
 '25' '3' 'None' '0' '0' '0' '0']


In [25]:
# We now need to write a one-hot encoder that can hanlde all possible permutations of game state. The one-hot-encoder
# must be able to encode all possible champion and item combinations.

# list : categories[i] holds the categories expected in the ith column. 
# The passed categories should not mix strings and numeric values within 
# a single feature, and should be sorted in case of numeric values.

categories = []
# First five categories is the shop, which can hold CHAMPION_IDs
categories += [champion_ids] * 5
# Next is the bench and board 

for i in range(18):
    categories += [['None'] + champion_ids] # the champion name
    categories += [['0'] + item_ids] * 3 # 3 items champ can hold
    categories.append(["0", "1","2","3"]) # champ levels

game_state_encoder = OneHotEncoder(
    categories=categories
)
game_state_encoder.fit([flattened_game_state])


one_hot_encoded_state = game_state_encoder.transform([flattened_game_state]).toarray()[0]
print("GAME STATE AFTER ENCODING", one_hot_encoded_state)
inverse_encoded = game_state_encoder.inverse_transform([one_hot_encoded_state])
print("GAME STATE INVERSE ENCODING", inverse_encoded)


GAME STATE AFTER ENCODING [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 